### Lanzcos algorithm

For eigenvalue problem of symmetric matrices. 

maintian a $v - \alpha_j q_j - \beta_j q_{j-1}$


In [ ]:
import numpy as np

def lanczos(A, b, k):
    n = len(b)
    Q = np.zeros((n, k+1))
    alpha = np.zeros(k)
    beta = np.zeros(k+1)

    Q[:, 0] = b / np.linalg.norm(b)

    v = np.zeros(n)

    for j in range(k):
        v = A @ Q[:, j]
        alpha[j] = np.dot(Q[:, j], v)
        v = v - alpha[j] * Q[:, j] - (beta[j] * Q[:, j-1] if j > 0 else 0)
        beta[j+1] = np.linalg.norm(v)

        if beta[j+1] == 0:
            break
        Q[:, j+1] = v / beta[j+1]

    return Q, alpha, beta

# test
A = np.array([[4., 1.], [1., 3.]])
b = np.array([1., 2.])

Q, alpha, beta = lanczos(A, b, k=2)
print("alpha:", alpha)
print("beta:", beta)


Ritz approximation:


eigenvalues of $T_{l}$

Cauchy interlace theorem shows that for ay i: $\lambda_i(T_k)$ increases monotonically with $k$: $\lambda_i(T_{k+1}) \ge \lambda_i(T_{k}) \ge \lambda_{i+1}(T_{k+1}) \ge \lambda_{i+1}(T_{k})$